In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [2]:
import os
import json
import pandas as pd
import time
from pinecone import Pinecone as pcn
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAI
from IPython.display import Markdown

/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### 01 Get API Keys

In [3]:
api_key = os.environ.get("PINECONE_API_KEY")
openai_api_key = os.getenv('OPENAI_API_KEY')

In [4]:
# configure client
pc = pcn(api_key=api_key)
index_name = 'recipes-index'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1106}},
 'total_vector_count': 1106}

### 02 Set Embedding Model

In [5]:
embed_model = "text-embedding-3-small"
embed = OpenAIEmbeddings(model=embed_model)

In [7]:
text_field = "description"

vectorStore = PineconeVectorStore(
  index, embed, text_field
)

### 03 Vector by Similarity Search

In [9]:
query = "Pain au chocolat"
query_res = vectorStore.similarity_search(query, k=3)

# Imprimir el metadata estructurado
for i, doc in enumerate(query_res):
    print(f"Document {i + 1}:")
    for key, value in doc.metadata.items():
        print(f"  {key}: {value}")
    print()

print(query_res)  # Para separar cada documento visualmente


Document 1:
  calories: 540
  carbs: 54g
  cook_time: 15 mins
  diet_type: Nut-free
  difficulty: Easy
  fat: 31g
  fibre: 4g
  id: 428
  ingredients: 420gplain flour, 8gsea salt, 60gcaster sugar, 220gunsalted butter(at least 82% fat works best), 20g softened, 20gfresh yeast, or 10g dried yeast, 1egg, beaten, 110mlwhole milk, 16dark chocolate batons
  instructions: Make the dough in astand mixer. Combine the flour, salt, sugar and 20g softened butter. Dissolve the yeast in around 80ml tepid water, around 38C, not too hot or it will kill the yeast. Add 20g of beaten egg and the milk to the stand mixer bowl. Pour in the yeast-water mixture and turn on to a low speed with a dough hook attachment for approximately 5 mins until the dough comes together to form a rough ball. Increase the speed to medium-high and knead for a further 6-10 mins until the dough is smooth. The exact time will depend on your stand mixer. Wrap the dough tightly and rest at room temperature for 10 mins. Unwrap and r

## 05 Llm completion

In [30]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
import pinecone
from langchain_core.prompts import ChatPromptTemplate

# Define la función de recuperación
def retrieve(query):
    # Recuperar contextos relevantes desde Pinecone
    query_res = vectorStore.similarity_search(query, k=3)
    contexts = []
    for i, doc in enumerate(query_res):
        # Convierte el diccionario de metadatos en una cadena de texto formateada
        metadata = doc.metadata
        formatted_context = (
            f"Title: {metadata.get('title', 'N/A')}\n"
            f"Description: {metadata.get('description', 'N/A')}\n"
            f"Preparation Time: {metadata.get('preparation Time', 'N/A')}\n"
            f"Cooking Time: {metadata.get('cook_time', 'N/A')}\n"
            f"Difficulty: {metadata.get('difficulty', 'N/A')}\n"
            f"Serves: {metadata.get('serves', 'N/A')}\n"
            f"Diet Type: {metadata.get('diet_type', 'N/A')}\n"
            f"Nutrition Facts: {metadata.get('calories', 'N/A')} | {metadata.get('carbs', 'N/A')} kcal | fat {metadata.get('fat', 'N/A')} | protein {metadata.get('protein', 'N/A')} | fibre {metadata.get('fibre', 'N/A')}\n"
            f"Ingredients: {metadata.get('ingredients', 'N/A')}\n"
            f"Instructions: {metadata.get('instructions', 'N/A')}\n"
        )
        contexts.append(formatted_context)
    
    print("context", contexts)
    prompt_contexts = "\n\n---\n\n".join(contexts) + "\n\n-----\n\n"
        # Construir el prompt con los contextos recuperados
    prompt_start = f"""
    Always add a kind and friendly message according to the context at the beginning of the response.
    Then follow this format:
    Title: [Title of the Recipe]
    Description: [Short Description of the Recipe]
    Preparation Time: [Preparation Time]
    Cooking Time: [Cooking Time]
    Difficulty: [Difficulty Level]
    Serves: [Number of Servings]
    Diet Type: [Diet Type]
    Nutrition: [List of Nutrition Facts]
    Ingredients: [List of Ingredients]
    Instructions: [Step-by-step Cooking Instructions]

    Answer the question based on the context below.\n\nContext:\n
    Recipes retrieved from Pinecone:
    {prompt_contexts}
    If you don't have any general information, just respond with "I don't know!"
    Make sure to omit any field where you don't have information."""

    if not contexts:
        return "No contexts retrieved. Try to answer the question yourself!"

    prompt_end = f"\n\nQuestion: {query}\nAnswer:"
    prompt = prompt_start + prompt_end

    print("###### prompt", prompt)
    return prompt

# Define la función de completar
def complete(prompt):
    # Inicializa el modelo OpenAI usando LangChain
    chat_model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    # Crear el formato de mensajes adecuado usando las clases SystemMessage y HumanMessage
    messages = [
        {"role": "system", "content": "You are a helpful assistant that always answers questions about recipes."},
        {"role": "user", "content": prompt}
    ]

    prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

    # Obtener la respuesta del modelo usando el método adecuado
    response = chat_model.generate(messages=messages)
    
    return response.generations[0].text.strip()

# Ejemplo de uso
query = "Pain au chocolat"
retrieved_prompt = retrieve(query)
answer = complete(retrieved_prompt)
print(answer)


context ['Title: Pain au chocolat\nDescription: N/A\nPreparation Time: N/A\nCooking Time: 15 mins\nDifficulty: Easy\nServes: Serves 8\nDiet Type: Nut-free\nNutrition Facts: 540 | 54g kcal | fat 31g | protein 8g | fibre 4g\nIngredients: 420gplain flour, 8gsea salt, 60gcaster sugar, 220gunsalted butter(at least 82% fat works best), 20g softened, 20gfresh yeast, or 10g dried yeast, 1egg, beaten, 110mlwhole milk, 16dark chocolate batons\nInstructions: Make the dough in astand mixer. Combine the flour, salt, sugar and 20g softened butter. Dissolve the yeast in around 80ml tepid water, around 38C, not too hot or it will kill the yeast. Add 20g of beaten egg and the milk to the stand mixer bowl. Pour in the yeast-water mixture and turn on to a low speed with a dough hook attachment for approximately 5 mins until the dough comes together to form a rough ball. Increase the speed to medium-high and knead for a further 6-10 mins until the dough is smooth. The exact time will depend on your stand 

TypeError: Got unknown type role